# Create_Modeling_Dataframes

This notebooks creates joins of dataframes useful for future modeling and exports them to .csv files.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%%time
path = '/Users/Jeff/Documents/Insight/Data'
aisles = pd.read_csv(path+'/instacart_2017_05_01/aisles.csv')
departments = pd.read_csv(path+'/instacart_2017_05_01/departments.csv')
orders = pd.read_csv(path+'/instacart_2017_05_01/orders.csv')
order_products__prior = pd.read_csv(path+'/instacart_2017_05_01/order_products__prior.csv')
order_products__train = pd.read_csv(path+'/instacart_2017_05_01/order_products__train.csv')
products = pd.read_csv(path+'/instacart_2017_05_01/products.csv')

CPU times: user 9.25 s, sys: 4.62 s, total: 13.9 s
Wall time: 14.3 s


## Update raw data tables with new columns

### Add organic or not to the products table

In [11]:
products = products[['product_id','product_name','aisle_id','department_id']]

# Function for adding a column for whether an item is organic or not:
def return_organic(prod_name):
    if 'organic' in prod_name.lower():
        return 1
    else:
        return 0

# Map function over products table
products['organic'] = products['product_name'].map(return_organic)

# Also merge aisle and department names to product table
products = products.merge(aisles,on='aisle_id')
products = products.merge(departments,on='department_id')
products.loc[:,'organic_produce'] = 0
products.loc[(products['organic']==1) & (products['department']=='produce') ,
         'organic_produce'] = 1
display(products.sample(20))
display(products.loc[products['organic_produce']==1].sample(10))


,product_id,product_name,aisle_id,department_id,organic,aisle,department,organic_produce
4326,6294,Peanut Caramel Candy Bar,45,19,0,candy chocolate,snacks,0
21216,9792,"Maca, Whole Root, Liquid Extract",47,11,0,vitamins supplements,personal care,0
39905,23150,Vegetable Classics 99% Fat Free Lentil Soup,69,15,0,soup broth bouillon,canned goods,0
26874,14903,Whips! Orange Creme Flavored Lowfat Yogurt Mousse,120,16,0,yogurt,dairy eggs,0
3714,29014,Whole Kernel Popcorn,23,19,0,popcorn jerky,snacks,0
15773,49198,Zero Soda,77,7,0,soft drinks,beverages,0
27210,31386,Peach Yogurt Protein Drink With a Crunchy Shot,120,16,0,yogurt,dairy eggs,0
43094,11286,Organic Uncured Beef Hot Dog,100,21,1,missing,missing,0
24892,20661,"Detox System Purifying Exfoliant Body Scrub, S...",25,11,0,soap,personal care,0
20165,18470,Severe Cold & Flu Formula Citrus Effervescent ...,11,11,0,cold flu allergy,personal care,0


,product_id,product_name,aisle_id,department_id,organic,aisle,department,organic_produce
41596,37607,Packaged Organic Blueberries,123,4,1,packaged vegetables fruits,produce,1
41173,2998,Potatoes Yellow Organic Bag,123,4,1,packaged vegetables fruits,produce,1
42552,31915,Organic Red Delicious Apple,24,4,1,fresh fruits,produce,1
42016,23165,Organic Leek,83,4,1,fresh vegetables,produce,1
41679,43961,Organic Peeled Whole Baby Carrots,123,4,1,packaged vegetables fruits,produce,1
42091,29662,Potato Yukon Gold Organic,83,4,1,fresh vegetables,produce,1
41784,3598,Organic White Button Mushrooms,83,4,1,fresh vegetables,produce,1
41675,43513,Organic Half Baby Spinach & Half Baby Arugula,123,4,1,packaged vegetables fruits,produce,1
42298,48775,Organic Red Cabbage,83,4,1,fresh vegetables,produce,1
42335,4121,Organic Red Delicious Apples,24,4,1,fresh fruits,produce,1


In [13]:
# Export a more full products table to csv
products.to_csv('products_organic.csv')

## Creating merges/joins of raw data tables


### Join product name and organic status to each products in orders_products


In [12]:
%%time
order_products = order_products__train # work on prior orders

order_products = order_products[['order_id','product_id','add_to_cart_order','reordered']]
order_products = order_products.merge(products,on='product_id')
display(order_products.head(20))

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,organic,aisle,department,organic_produce
0,1,49302,1,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
1,816049,49302,7,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
5,2445303,49302,2,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
6,2853065,49302,12,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
7,3231517,49302,6,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,0
8,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,1,other creams cheeses,dairy eggs,0
9,68474,11109,8,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,1,other creams cheeses,dairy eggs,0


CPU times: user 394 ms, sys: 216 ms, total: 611 ms
Wall time: 629 ms


### Get the organic status of each order

In [13]:
# Sum all the organic products in each order:
orders_organic = order_products[['order_id','organic','organic_produce']].groupby(['order_id']).sum()
orders_organic.sample(5)

,organic,organic_produce
order_id,,
2868483,5,2
166024,1,1
1334696,2,1
134570,1,0
2167990,2,1


### Getting info about a user's most recent order and the items therein (e.g., whether it has an organic item or not)
In preparation to join to previous order characteristics

In [14]:
%%time

# Join previous order_ids by user to each order.

# Create custom column of user_id and order_number separated by a decimal place
orders['user_id_order'] = orders['user_id']+0.001*orders['order_number']
orders['user_id_prevorder'] = orders['user_id']+0.001*(orders['order_number']-1)
# display(orders.head(13))

# Use this column to join each order to its previous order and get the order_id
orders = orders.merge(orders[['order_id','user_id_order']].set_index('user_id_order'),
                     left_on='user_id_prevorder',right_on='user_id_order',
                     how='left',suffixes=('','_prev'))

# orders.loc[:,'order_id_prev'] = orders['order_id_prev'].astype('int64') #can't do this because some are NaNs
display(orders.head(13))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev
0,2539329,1,prior,1,2,8,NaN,1.001,1.000,NaN
1,2398795,1,prior,2,3,7,15.0,1.002,1.001,2539329.0
2,473747,1,prior,3,3,12,21.0,1.003,1.002,2398795.0
3,2254736,1,prior,4,4,7,29.0,1.004,1.003,473747.0
4,431534,1,prior,5,4,15,28.0,1.005,1.004,2254736.0
5,3367565,1,prior,6,2,7,19.0,1.006,1.005,431534.0
6,550135,1,prior,7,1,9,20.0,1.007,1.006,3367565.0
7,3108588,1,prior,8,1,14,14.0,1.008,1.007,550135.0
8,2295261,1,prior,9,1,16,0.0,1.009,1.008,3108588.0
9,2550362,1,prior,10,4,8,30.0,1.010,1.009,2295261.0


CPU times: user 10.5 s, sys: 1.59 s, total: 12.1 s
Wall time: 12.7 s


Last column (`order_id_prev`) matches first column (`order_id`) one row up.

In [15]:
# Join order organic status to orders table. Can use df.join() here because index is order_id above
orders_by_o = orders.join(orders_organic,on='order_id',rsuffix='2')

display(orders_by_o.loc[orders_by_o['eval_set']=='train'].head(20)) # check that join worked
# display(orders_by_o.head(20)) # for when original orders_products table is for all data, not just training data.

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,organic,organic_produce
10,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,3.0,0.0
25,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,6.0,1.0
49,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,5.0,3.0
74,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,1.0,0.0
78,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,9.0,7.0
82,1094988,9,train,4,6,10,30.0,9.004,9.003,1830137.0,6.0,1.0
88,1822501,10,train,6,0,19,30.0,10.006,10.005,1353310.0,0.0,0.0
115,1827621,13,train,13,0,21,8.0,13.013,13.012,1789302.0,1.0,0.0
129,2316178,14,train,14,2,19,11.0,14.014,14.013,3394109.0,0.0,0.0
200,2180313,17,train,41,3,10,30.0,17.041,17.040,3350883.0,0.0,0.0


In [24]:
# Adding binary indicators for whether the order had organic items or not:

# This is creating order-level vs product-level flags. 
# i.e., categorize each order by whether or not it had an organic item or not, and see if we can predict organic carts.

# Create new boolean columns
orders_by_o['any_organic'] = orders_by_o['organic']>0
orders_by_o['any_organic_produce'] = orders_by_o['organic_produce']>0
orders_by_o['organic_non_produce'] = orders_by_o['organic'] - orders_by_o['organic_produce']
orders_by_o['any_organic_non_produce'] = orders_by_o['organic_non_produce']>0

### Do same for previous orders

In [28]:
# Filter table to get only previous orders
orders_products_prev = order_products__prior.loc[order_products__prior['order_id'].isin(users_order_most_recent['order_id_prev'].values)]

# Merge with products table to get same details about previous order products
orders_products_prev = orders_products_prev[['order_id','product_id','add_to_cart_order','reordered']]
orders_products_prev = orders_products_prev.merge(products,on='product_id')

### Merging most previous past order's organic status is to most recent order,

In [17]:
# Start with the most recent order by user

users_order_most_recent = orders_by_o.loc[orders_by_o['eval_set'].map(lambda x: x in ['train'])]
display(len(users_order_most_recent)) #206209 users total, 131209 for train
# users_order_most_recent['user_id'].value_counts() #only one of every user, as expected.
display(users_order_most_recent.sort_values('user_id').head(10))

131209

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,organic,organic_produce,any_organic,any_organic_produce,organic_non_produce,any_organic_non_produce
10,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,3.0,0.0,True,False,3.0,True
25,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,6.0,1.0,True,True,5.0,True
49,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,5.0,3.0,True,True,2.0,True
74,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,1.0,0.0,True,False,1.0,True
78,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,9.0,7.0,True,True,2.0,True
82,1094988,9,train,4,6,10,30.0,9.004,9.003,1830137.0,6.0,1.0,True,True,5.0,True
88,1822501,10,train,6,0,19,30.0,10.006,10.005,1353310.0,0.0,0.0,False,False,0.0,False
115,1827621,13,train,13,0,21,8.0,13.013,13.012,1789302.0,1.0,0.0,True,False,1.0,True
129,2316178,14,train,14,2,19,11.0,14.014,14.013,3394109.0,0.0,0.0,False,False,0.0,False
200,2180313,17,train,41,3,10,30.0,17.041,17.040,3350883.0,0.0,0.0,False,False,0.0,False


In [20]:
# Also Categorize previous orders by organic status:

# Add organic status to previous order table
orders_prev_organic = orders_products_prev[['order_id','organic']].groupby(['order_id']).sum()
orders_prev_organic_produce = orders_products_prev[['order_id','organic_produce']].groupby(['order_id']).sum()

In [21]:
# Use users_order_most_recent, then join to previous order, and info about that order (organic or not)

users_order_by_prev_org = users_order_most_recent.merge(orders_prev_organic,
                                                        left_on='order_id_prev',right_on='order_id',
                                                        suffixes=('','_prev'))
users_order_by_prev_org = users_order_by_prev_org.merge(orders_prev_organic_produce,
                                                        left_on='order_id_prev',right_on='order_id',
                                                        suffixes=('','_prev'))
users_order_by_prev_org['any_organic_prev'] = users_order_by_prev_org['organic_prev']>0
users_order_by_prev_org['any_organic_produce_prev'] = users_order_by_prev_org['organic_produce_prev']>0
display(users_order_by_prev_org)
# display(users_order_by_prev_org.columns)
# display(users_order_by_prev_org.loc[users_order_by_prev_org['order_id_prev']==25])



,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,organic,organic_produce,any_organic,any_organic_produce,organic_non_produce,any_organic_non_produce,organic_prev,organic_produce_prev,any_organic_prev,any_organic_produce_prev
0,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,3.0,0.0,True,False,3.0,True,2,0,True,False
1,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,6.0,1.0,True,True,5.0,True,4,3,True,True
2,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,5.0,3.0,True,True,2.0,True,6,3,True,True
3,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,1.0,0.0,True,False,1.0,True,5,3,True,True
4,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,9.0,7.0,True,True,2.0,True,4,2,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,206199.020,206199.019,1932632.0,1.0,1.0,True,True,0.0,False,2,0,True,False
131205,943915,206200,train,24,6,19,6.0,206200.024,206200.023,1909878.0,10.0,7.0,True,True,3.0,True,20,13,True,True
131206,2371631,206203,train,6,4,19,30.0,206203.006,206203.005,133582.0,4.0,2.0,True,True,2.0,True,7,1,True,True
131207,1716008,206205,train,4,1,16,10.0,206205.004,206205.003,414137.0,8.0,4.0,True,True,4.0,True,4,1,True,True


In [18]:
# Export for use in modeling
users_order_by_prev_org.to_csv('../modeling_dfs/users_order_by_prev_org.csv')

### Merging the most recent order to the previous order's items by department/aisle

In [33]:
%%time

sample_order_id = 25 # Use this sample row to confirm that the table manipulations worked

# Get aisles for previous orders
order_by_aisle = pd.DataFrame(orders_products_prev.groupby(['order_id'])['aisle'].value_counts())
display(order_by_aisle.head(5))

# Use .unstack() function to make a wide dataframe of the number of items from each aisle by order
order_by_aisle_wide = order_by_aisle.unstack(fill_value=0)
display(order_by_aisle_wide.head(5))
# Check that the unstacking worked:
# display(order_by_aisle_wide.loc[sample_order_id].sort_values(ascending=False).head(5)) #order_id=1

# Get departments for previous orders
order_by_dept = pd.DataFrame(orders_products_prev.groupby(['order_id'])['department'].value_counts())
display(order_by_dept.head(5))
order_by_dept = order_by_dept.unstack(fill_value=0)
display(order_by_dept.head(5))


aisle
order_id aisle                         
25       packaged cheese              3
         popcorn jerky                2
         dog food care                1
         eggs                         1
         frozen appetizers sides      1

aisle                               \
aisle    air fresheners candles asian foods baby accessories   
order_id                                                       
25                            0           0                0   
40                            0           0                0   
43                            0           0                0   
83                            0           0                0   
97                            0           0                0   

                                                                \
aisle    baby bath body care baby food formula bakery desserts   
order_id                                                         
25                         0                 0               0   
40                         0                 0               0   
43                         0                 0               0   
83                         0                 0               0   
97                         0                 0               0   

                                                                        ...  \
aisle    baking ingredients baking supplies decor beauty beers coolers  ...   
order_id                                                                ...   
25                        0                     0      0             0  ...   
40                        0                     0      0             0  ...   
43                        0                     0      0             0  ...   
83                        0                     0      0             0  ...   
97                        0                     0      0             0  ...   

                                                                  \
aisle    spreads tea tofu meat alternatives tortillas flat bread   
order_id                                                           
25             0   0                      0                    0   
40             0   0                      0                    0   
43             0   0                      0                    0   
83             0   0                      0                    0   
97             1   0                      0                    0   

                                                                     \
aisle    trail mix snack mix trash bags liners vitamins supplements   
order_id                                                              
25                         0                 0                    0   
40                         0                 0                    0   
43                         0                 1                    0   
83                         0                 0                    0   
97                         0                 0                    0   

                                                           
aisle    water seltzer sparkling water white wines yogurt  
order_id                                                   
25                                   0           0      0  
40                                   1           0      0  
43                                   0           0      0  
83                                   0           0      0  
97                                   0           0      4  

[5 rows x 134 columns]

department
order_id department              
25       frozen                 5
         dairy eggs             4
         snacks                 2
         meat seafood           1
         pets                   1

department                                                      \
department    alcohol babies bakery beverages breakfast bulk canned goods   
order_id                                                                    
25                  0      0      0         0         0    0            0   
40                  0      0      0         1         0    0            0   
43                  0      0      0         0         0    0            0   
83                  0      0      0         1         1    0            0   
97                  0      0      1         2         1    0            1   

                                            ...                          \
department dairy eggs deli dry goods pasta  ... household international   
order_id                                    ...                           
25                  4    0               0  ...         0             0   
40                  2    0               0  ...         0             0   
43                  3    0               0  ...         1             0   
83                  1    0               1  ...         0             0   
97                  4    1               1  ...         1             0   

                                                                                
department meat seafood missing other pantry personal care pets produce snacks  
order_id                                                                        
25                    1       0     0      0             0    1       1      2  
40                    0       0     0      0             0    0       0      0  
43                    0       0     0      0             1    0       1      1  
83                    0       0     0      1             0    0       4      2  
97                    0       0     0      2             7    0       0      2  

[5 rows x 21 columns]

CPU times: user 2.73 s, sys: 400 ms, total: 3.13 s
Wall time: 2.48 s


In [31]:
%%time

# Use users_order_most_recent, then join to previous order, and info about that order (# aisles, depts)

users_order_by_prev_dept = users_order_most_recent.merge(order_by_dept,left_on='order_id_prev',right_on='order_id')
display(users_order_by_prev_dept)
# display(users_order_by_prev_dept.columns)
# display(users_order_by_prev_dept.loc[users_order_most_recent['order_id_prev']==25])

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,...,"(department, household)","(department, international)","(department, meat seafood)","(department, missing)","(department, other)","(department, pantry)","(department, personal care)","(department, pets)","(department, produce)","(department, snacks)"
0,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,...,0,0,0,0,0,0,0,0,0,3
1,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,...,0,1,0,0,0,1,0,0,4,1
2,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,...,0,1,0,0,0,0,0,0,5,0
3,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,...,0,0,1,0,0,0,0,0,4,1
4,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,...,0,0,0,0,0,1,0,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,206199.020,206199.019,1932632.0,...,4,0,0,2,0,1,0,0,1,9
131205,943915,206200,train,24,6,19,6.0,206200.024,206200.023,1909878.0,...,0,0,1,0,0,6,0,0,16,1
131206,2371631,206203,train,6,4,19,30.0,206203.006,206203.005,133582.0,...,1,0,0,0,0,1,0,0,1,7
131207,1716008,206205,train,4,1,16,10.0,206205.004,206205.003,414137.0,...,0,0,0,0,0,0,0,0,3,0


CPU times: user 305 ms, sys: 76.2 ms, total: 381 ms
Wall time: 314 ms


In [32]:
# Export for use in modeling
users_order_by_prev_dept.to_csv('../modeling_dfs/users_order_by_prev_dept.csv')

### Export shopping history by user

In [7]:
%%time

# For each user, get the products they've purchased in prior

# Get prior orders by user
users_order_prior = orders.loc[orders['eval_set'].map(lambda x: x in ['prior'])]

# Merge products in prior orders to the user for each order
print(len(order_products__prior))
display(order_products__prior.head(5))
user_order_products_prior_train = order_products__prior.merge(users_order_prior[['order_id','user_id']],
                                                        on='order_id')
print(len(user_order_products_prior_train))
display(user_order_products_prior_train.head(5))

# Check join worked
# display(users_order_prior.loc[users_order_prior['order_id']==2]) # does this return user 202279?

# Group products ordered by user
user_products_freq_train_long = user_order_products_prior_train[['user_id','product_id']]\
                                .groupby(['user_id','product_id']).size()
# display(user_products_freq_train_long.head(25))

# Reset index to remove MultiIndex
user_products_freq_train_long = user_products_freq_train_long.reset_index()
user_products_freq_train_long.columns = ['user_id','product_id','freq']
display(user_products_freq_train_long.head(25))

# Group order_products__prior by user_id, and store each list of unique products as a dictionary with "user_id" as key.
# (Testing for building colab filter)
prods_dict = dict(user_products_freq_train_long.groupby('user_id')['product_id'].apply(list))
print('Ordered products for User 1: ',prods_dict[1])
# user_products_freq_train_long.loc[user_products_freq_train_long['user_id']==1] #check dictionary result is as expected


32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


32434489


,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


,user_id,product_id,freq
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3
5,1,13176,2
6,1,14084,1
7,1,17122,1
8,1,25133,8
9,1,26088,2


Ordered products for User 1:  [196, 10258, 10326, 12427, 13032, 13176, 14084, 17122, 25133, 26088, 26405, 30450, 35951, 38928, 39657, 41787, 46149, 49235]
CPU times: user 51.9 s, sys: 15.7 s, total: 1min 7s
Wall time: 1min 10s


In [8]:
%%time

# Export organic produce history

# Merge users past shopping history with product table which has organic and organic produce status
user_products_org = user_products_freq_train_long.merge(products,on='product_id')
display(user_products_org.head(5))

# Double-check merge
display(products.loc[products['product_id']==196]) # should be soda

users_hist_org = user_products_org[['user_id','organic','organic_produce']].groupby('user_id').sum()
display(users_hist_org)

# Export
users_hist_org.to_csv('../modeling_dfs/users_hist_org.csv')

,user_id,product_id,freq,product_name,aisle_id,department_id,organic,aisle,department,organic_produce
0,1,196,10,Soda,77,7,0,soft drinks,beverages,0
1,15,196,5,Soda,77,7,0,soft drinks,beverages,0
2,19,196,3,Soda,77,7,0,soft drinks,beverages,0
3,21,196,1,Soda,77,7,0,soft drinks,beverages,0
4,31,196,2,Soda,77,7,0,soft drinks,beverages,0


,product_id,product_name,aisle_id,department_id,organic,aisle,department,organic_produce
15316,196,Soda,77,7,0,soft drinks,beverages,0


,organic,organic_produce
user_id,,
1,6,2
2,28,9
3,8,5
4,2,2
5,12,7
...,...,...
206205,8,2
206206,56,22
206207,43,17


CPU times: user 7.6 s, sys: 5.32 s, total: 12.9 s
Wall time: 13.3 s


## Other avenues tried:

### Exporting whether an order has a top50 organic item or not, based on aisles/departments of the order.
For each order, join to info about that order (aisles, depts). Eventually, we want to predict each column e.g., 13176 (organic bananas.)

In [18]:
# Get top 50 items by frequency
organic_top50 = order_products.loc[order_products['organic']==1,'product_name'].value_counts()\
                    .head(50).index.values
display(organic_top50)
organic_top50_ids = order_products.loc[order_products['organic']==1,'product_id'].value_counts()\
                    .head(50).index.values
display(organic_top50_ids)

array(['Bag of Organic Bananas', 'Organic Strawberries',
       'Organic Baby Spinach', 'Organic Avocado', 'Organic Hass Avocado',
       'Organic Raspberries', 'Organic Blueberries', 'Organic Whole Milk',
       'Organic Cucumber', 'Organic Zucchini', 'Organic Yellow Onion',
       'Organic Garlic', 'Organic Grape Tomatoes', 'Organic Red Onion',
       'Organic Baby Carrots', 'Organic Cilantro', 'Organic Lemon',
       'Organic Fuji Apple', 'Organic Baby Arugula',
       'Organic Large Extra Fancy Fuji Apple', 'Organic Blackberries',
       'Organic Gala Apples', 'Organic Half & Half',
       'Michigan Organic Kale', 'Organic Small Bunch Celery',
       'Organic Garnet Sweet Potato (Yam)', 'Organic Tomato Cluster',
       'Organic Peeled Whole Baby Carrots',
       'Organic Italian Parsley Bunch', 'Organic Red Bell Pepper',
       'Organic Granny Smith Apple', 'Organic Banana',
       'Apple Honeycrisp Organic', 'Organic Unsweetened Almond Milk',
       'Organic Ginger Root', 'Organic

array([13176, 21137, 21903, 47766, 47209, 27966, 39275, 27845, 30391,
       45007, 22935, 24964, 40706,  8518, 42265, 31717,  5876, 28204,
       21616, 19057, 26604, 37646, 49235, 28985, 44359, 48679, 41950,
       43961, 34126, 10749, 39877, 37067,  8277, 35951, 46667, 22035,
        8174, 22825, 39928, 18465, 27521, 16759,  9839, 27156,  5785,
       37687, 20995, 34243,  7781, 43789])

In [25]:
# Get whether each order has one of the top 50 items
each_order = orders_by_o.loc[orders_by_o['eval_set'].map(lambda x: x in ['train'])] #train for now, but eventually all
display(each_order.head())
display(len(each_order))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,organic,organic_produce,any_organic,any_organic_produce,organic_non_produce,any_organic_non_produce
10,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,3.0,0.0,True,False,3.0,True
25,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,6.0,1.0,True,True,5.0,True
49,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,5.0,3.0,True,True,2.0,True
74,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,1.0,0.0,True,False,1.0,True
78,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,9.0,7.0,True,True,2.0,True


131209

In [26]:
# add dummy columns for each of the top 50 organic foods so we can do a groupby.sum of these columns
order_products_top50 = order_products
for organic_top50_id in list(organic_top50_ids):
    order_products_top50[organic_top50_id] = order_products_top50['product_id']==organic_top50_id
    order_products_top50[organic_top50_id] = order_products_top50[organic_top50_id].astype(int)

display(order_products_top50.head())
display(order_products_top50.loc[order_products_top50['product_id']==27521].head()) #column 27521 is 1.

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,organic,aisle,department,...,27521,16759,9839,27156,5785,37687,20995,34243,7781,43789
0,1,49302,1,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,...,0,0,0,0,0,0,0,0,0,0
1,816049,49302,7,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,...,0,0,0,0,0,0,0,0,0,0
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,...,0,0,0,0,0,0,0,0,0,0
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,...,0,0,0,0,0,0,0,0,0,0
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,0,yogurt,dairy eggs,...,0,0,0,0,0,0,0,0,0,0


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,organic,aisle,department,...,27521,16759,9839,27156,5785,37687,20995,34243,7781,43789
286926,1468,27521,7,1,Organic Lacinato (Dinosaur) Kale,83,4,1,fresh vegetables,produce,...,1,0,0,0,0,0,0,0,0,0
286927,3484,27521,21,1,Organic Lacinato (Dinosaur) Kale,83,4,1,fresh vegetables,produce,...,1,0,0,0,0,0,0,0,0,0
286928,4468,27521,3,1,Organic Lacinato (Dinosaur) Kale,83,4,1,fresh vegetables,produce,...,1,0,0,0,0,0,0,0,0,0
286929,4680,27521,7,1,Organic Lacinato (Dinosaur) Kale,83,4,1,fresh vegetables,produce,...,1,0,0,0,0,0,0,0,0,0
286930,4827,27521,16,1,Organic Lacinato (Dinosaur) Kale,83,4,1,fresh vegetables,produce,...,1,0,0,0,0,0,0,0,0,0


In [21]:
# for each top50_organic item, sum all of that item in each order and merge to each_order df
orders_top50_id = order_products_top50[['order_id']+list(organic_top50_ids)].groupby(['order_id']).sum()
display(orders_top50_id)

,13176,21137,21903,47766,47209,27966,39275,27845,30391,45007,...,27521,16759,9839,27156,5785,37687,20995,34243,7781,43789
order_id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3421056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3421058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
%%time

sample_order_id = 3421056 # check this order

# get aisles and departments for orders in train
order_by_aisle = pd.DataFrame(order_products.groupby(['order_id'])['aisle'].value_counts())
display(order_by_aisle.head(5))

order_by_aisle = order_by_aisle.unstack(fill_value=0)
display(order_by_aisle.head(5))
#check that the unstacking worked:
# display(order_by_aisle.loc[sample_order_id].sort_values(ascending=False).head(10)) #order_id=1

#repeat with 21 departments
order_by_dept = pd.DataFrame(order_products.groupby(['order_id'])['department'].value_counts())
display(order_by_dept.head(5))

order_by_dept = order_by_dept.unstack(fill_value=0)
display(order_by_dept.head(5))
#check that the unstacking worked:
# display(order_by_dept.loc[sample_order_id].sort_values(ascending=False).head(10)) #order_id=1


aisle
order_id aisle                      
1        fresh fruits              2
         fresh vegetables          2
         canned meat seafood       1
         other creams cheeses      1
         packaged cheese           1

aisle                               \
aisle    air fresheners candles asian foods baby accessories   
order_id                                                       
1                             0           0                0   
36                            0           0                0   
38                            0           0                0   
96                            0           0                0   
98                            0           0                0   

                                                                \
aisle    baby bath body care baby food formula bakery desserts   
order_id                                                         
1                          0                 0               0   
36                         0                 0               0   
38                         0                 0               0   
96                         0                 0               0   
98                         0                 0               0   

                                                                        ...  \
aisle    baking ingredients baking supplies decor beauty beers coolers  ...   
order_id                                                                ...   
1                         0                     0      0             0  ...   
36                        0                     0      0             0  ...   
38                        0                     0      0             0  ...   
96                        0                     0      0             0  ...   
98                        2                     0      0             0  ...   

                                                                  \
aisle    spreads tea tofu meat alternatives tortillas flat bread   
order_id                                                           
1              0   0                      0                    0   
36             0   0                      0                    0   
38             0   0                      0                    0   
96             0   0                      0                    0   
98             0   0                      0                    1   

                                                                     \
aisle    trail mix snack mix trash bags liners vitamins supplements   
order_id                                                              
1                          0                 0                    0   
36                         0                 0                    0   
38                         0                 0                    0   
96                         0                 0                    0   
98                         0                 0                    0   

                                                           
aisle    water seltzer sparkling water white wines yogurt  
order_id                                                   
1                                    0           0      1  
36                                   1           0      0  
38                                   0           0      0  
96                                   0           0      0  
98                                   1           0      1  

[5 rows x 134 columns]

department
order_id department              
1        produce                4
         dairy eggs             3
         canned goods           1
36       dairy eggs             3
         produce                3

department                                                      \
department    alcohol babies bakery beverages breakfast bulk canned goods   
order_id                                                                    
1                   0      0      0         0         0    0            1   
36                  0      0      0         1         0    0            0   
38                  0      0      0         0         0    0            0   
96                  0      0      0         0         0    0            0   
98                  0      0      1         4         0    0            8   

                                            ...                          \
department dairy eggs deli dry goods pasta  ... household international   
order_id                                    ...                           
1                   3    0               0  ...         0             0   
36                  3    1               0  ...         0             0   
38                  0    0               0  ...         0             0   
96                  0    1               0  ...         0             0   
98                 11    3               0  ...         5             0   

                                                                                
department meat seafood missing other pantry personal care pets produce snacks  
order_id                                                                        
1                     0       0     0      0             0    0       4      0  
36                    0       0     0      0             0    0       3      0  
38                    0       0     0      1             0    0       6      1  
96                    0       0     0      0             0    0       4      0  
98                    1       0     0      4             1    0       7      1  

[5 rows x 21 columns]

CPU times: user 2.98 s, sys: 363 ms, total: 3.34 s
Wall time: 2.31 s


In [28]:
%%time

# Join each order columns to top50 organic status, aisles, and depts of that order
each_order = each_order[['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'organic',
       'organic_produce', 'any_organic', 'any_organic_produce',
       'organic_non_produce', 'any_organic_non_produce']]
each_order = each_order.join(orders_top50_id, #can use join because orders_top50_id Index is order_id
                             on='order_id')
each_order = each_order.join(order_by_aisle, #can use join because orders_top50_id Index is order_id
                             on='order_id')
each_order = each_order.join(order_by_dept, #can use join because orders_top50_id Index is order_id
                             on='order_id')

display(each_order.loc[each_order['order_id']==3421056])

/Users/Jeff/opt/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/Jeff/opt/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexes/base.py:3004: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  indexer = self._engine.get_indexer(target._ndarray_values)
/Users/Jeff/opt/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexes/base.py:4232: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self._engine


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,organic,organic_produce,any_organic,...,"(department, household)","(department, international)","(department, meat seafood)","(department, missing)","(department, other)","(department, pantry)","(department, personal care)","(department, pets)","(department, produce)","(department, snacks)"
1395689,3421056,83898,train,13,2,20,11.0,0.0,0.0,False,...,0,0,0,0,0,1,0,0,0,0


CPU times: user 1.07 s, sys: 509 ms, total: 1.58 s
Wall time: 1.14 s


In [29]:
# Export
each_order.to_csv('../modeling_dfs/orders_by_top50organic_by_aisle_by_dept.csv')

## Other avenues to pursue in the future:

### - Join a user's most recent order to the 2 most recent orders.
Use order_by_o, then join to previous order, and info about that order (# aisles, depts, %organic) to predict organic_sum, or any_organic.

### - Same as above, but for 3 most recent orders

### - Predicting users that make the switch to buying organic. 
Need to categorize each order by whether or not it had an organic item or not, and, if it had an organic item, whether the equivalent non-organic item was purchased previously or not.

For this, need to check all prior orders for a specific id. To start, may have to narrow down to specific items first, e.g., Bananas.

### - Predicting orders with organic X. (Like above but more targeted)

**Order-level:** Need to categorize each order by specific organic products (e.g., bananas) and compare to other orders which contain normal bananas.
--> Predict what characteristics make people have organic bananas, in an effort to target users for switching.

There may be interesting differences between types of produces. E.g., pesticide residue persists on some types of fruits versus other, so maybe people want to buy organic fruits that don't have a peel
For organic meat, would there be a fear of growth hormones or antibiotics?